In [3]:
import time
import math
import sympy as sp
import numpy as np
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *

In [4]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler 

In [5]:
# Rejection sampling algorithm 
# If parameters chosen as BBC+18, output is 1 with probability 1/rho

def Rej(Z, B, sigma, rho):
    u = np.random.random()
    if u > 1/rho * np.exp((-2*Z.flatten().dot(B.flatten()) + np.linalg.norm(B.flatten())**2)/(2*sigma**2)):
        return 0
    else:
        return 1

In [6]:
#Balance algorithm
#Given a matrix x with coeffs in [0, q], returns the same matrix with coeffs in [-(q-1)/2, (q-1)/2]
def balance(x, q):
    return np.array([n if n <= (q-1)/2 else n-q for n in x.flatten()]).reshape(np.shape(x))

In [7]:
class Prover:
    
    A = None
    S = None
    T = None
    sigma = None
    Y = None
     
    def __init__(self, lamb, q, A, S, T):
        self.q = q
        self.A = A
        self.S = S
        self.T = T
        self.Y = []
        
        s = np.linalg.norm(S, 2) + 1  # Upper bound on s_1(S)
        
        self.l = S.shape[1]
        self.n = lamb + 2
        assert A.shape[1] == S.shape[0]
        self.v = A.shape[1]
        
        self.rho = 3
        self.sigma = 12/np.log(self.rho)*s*np.sqrt(self.l*self.n)
        self.B = math.sqrt(2*self.v)*self.sigma  # This is somehow known by the verifier
 
        # print("s = " + str(s))
        # print("sigma = " + str(self.sigma))
        # print("B = " + str(self.B))
        
    
    def calculateW(self, n):
        D = DiscreteGaussianDistributionIntegerSampler(self.sigma)
        self.Y = np.array([D() for _ in range(self.v*self.n)]).reshape(self.v,self.n)
        W = np.matmul(self.A, self.Y)
        return W
    
    def calculateZ(self, C):
        self.C = C
        self.Z = np.matmul(self.S, self.C) + self.Y
        return self.Z
    
    def reject(self):
        def Rej(Z, B, sigma, rho):
            u = np.random.random()
            if u > 1/rho * np.exp((-2*Z.flatten().dot(B.flatten()) + np.linalg.norm(B.flatten())**2)/(2*sigma**2)):
                return 0
            else:
                return 1
        return Rej(self.Z, self.S@self.C, self.sigma, self.rho)

In [8]:
class Verifier:
    
    A = None
    T = None
    W = None
    C = None
    
    def __init__(self, A, T):
        self.A = A
        self.T = T
        self.W = []
        self.C = []
        
    def calculateC(self, W):
        self.W = W
        n = W.shape[1]
        l = self.T.shape[1]
        self.C = np.random.randint(2, size=[l,n])
        return self.C
    
    def verify(self, Z, B, q):
        AZ = self.A @ Z
        TC = self.T @ self.C
        return np.array_equal(AZ % q, (TC + self.W) % q) and np.all(np.linalg.norm(Z, np.inf, axis=0) <= B)
        

In [9]:
class Proof:
    
    prover = None
    verifier = None
    
    W = None
    C = None
    Z = None
    
    bit = None
    num_aborts = None
    running_time = None
    proof_size = None
    
    
    def __init__(self, lamb, q, A, S, T):
        self.prover = Prover(lamb, q, A, S, T)
        self.verifier = Verifier(A, T)

    def run(self):
        abort = True
        self.num_aborts = 0

        # Protocol starts

        start_time = time.time()

        self.proof_size = 0

        self.W = self.prover.calculateW(self.prover.n)
        
        self.proof_size += self.W.nbytes

        while abort:

            self.C = self.verifier.calculateC(self.W)

            self.Z = self.prover.calculateZ(self.C)

            self.proof_size += self.Z.nbytes #  + self.C.nbytes

            #Rejection sampling

            abort = self.prover.reject()
            self.num_aborts += abort

        #Verification

        B = self.prover.B  # ???

        self.bit = self.verifier.verify(self.Z, B, self.prover.q)

        end_time = time.time()
        
        self.running_time = end_time - start_time
        
        return self.bit
        
    def print_stats(self):

        print("Verification: " +  str(self.bit))
        print("Times aborted: " + str(self.num_aborts))
        print("Total execution time: " + str(self.running_time) + " seconds")
        print("Size of the witness: " + str(self.prover.S.nbytes / 1000) + " kB")
        print("Size of the proof: " + str(self.proof_size / 1000) + " kB")


In [11]:
def ZKP_test():
    
    lamb = 128 # Security parameter lambda
    q = sp.nextprime(4099)  # Prime for base field Z_q

    ############# Invent matrices A, S, T ###################
    l = 11 #Number of equations
    r = 7 #poly(lambda)
    v = 231 #poly(lambda)  # Parameters r, v and l should be chosen by the commitment scheme given N = l*r*v

    A = np.random.randint(0, q, size=[r, v])
    S = np.random.randint(-1, 2, size=[v, l])
    T = (A @ S) % q

    #########################################################
    
    # print(A)
    # print(S)
    # print(T)

    proof = Proof(lamb, q, A, S, T)
    proof.run()
    proof.print_stats()

ZKP_test()

Verification: True
Times aborted: 1
Total execution time: 0.07257461547851562 seconds
Size of the witness: 20.328 kB
Size of the proof: 487.76 kB


In [166]:
def rejSamplingTest():
    res = 0
    r = 10
    n = 10
    rho = 3
    times = 20000
    print("Should abort " + str(100/rho) + "% of the time")
    for _ in range (times):
        B = np.random.randint(0, 2**32, size = [r,n])
        sigma = 12/math.log(rho) * np.linalg.norm(B,2)
        D = DiscreteGaussianDistributionIntegerSampler(sigma)
        Y = np.array([D() for _ in range(r*n)]).reshape(r,n)
        res += Rej(Y+B, B, sigma, rho)
    print("Aborted " + str(100*res/times) + "% of the time")
    
rejSamplingTest()

Should abort 33.333333333333336% of the time


/home/mak/Downloads/SageMath/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in long_scalars
  


Aborted 32.445% of the time
